# Independent Sample t-Test and A/B Test

## Theory

**Assumptions**:
1. X and Y are independent
1. X and Y have same variance $\sigma^2$
1. X and Y from normal distribution, respectively




**(*)Test Statistic Under Un-equal Variance**
$$T=\dfrac{(\bar{X}-\bar{Y})-(\mu_X-\mu_Y)}{\sqrt{\dfrac{S^2_X}{N_X}+\dfrac{S^2_Y}{N_Y}}} \sim t\ (df)$$

$$df=\dfrac{\left(\dfrac{s^2_X}{N_Y}+\dfrac{s^2_Y}{N_Y}\right)^2}{\dfrac{(s^2_X/N_X)^2}{N_X-1}+\dfrac{(s^2_Y/N_Y)^2}{N_Y-1}}$$


**Test Statistic Under Equal Variance**
- Under Normal Distribution (***Independent Sample T-test***):

    $$\hat \sigma_X = S_X,\hat \sigma_Y = S_Y $$
    
$$T = \frac{(\bar X - \bar Y) - (\mu_X - \mu_Y)}{\hat \sigma _{\bar X - \bar Y}} \sim \ t\ (df=N_X+N_Y-2)$$

$$\hat \sigma _{\bar X - \bar Y} = {s_{\bar X - \bar Y}} = s_p\sqrt{\frac{1}{N_X}+\frac{1}{N_Y}} \xrightarrow{N_X = N_Y}  s_p \sqrt \frac{2}{N}$$


$$unbiased:\  s_p^2 = \frac{(N_X-1)\hat \sigma_X^2 + (N_Y-1)\hat \sigma_Y^2}{N_X+N_Y-2}\xrightarrow{N_X = N_Y} \frac{1}{2}(\sigma_X^2 + \sigma_Y^2)\xrightarrow{\sigma_X^2 = \sigma_Y^2} \sigma^2$$


    


- Under Binomial Distribution (***A/B Test***): 
    - Option 1: Assume same proportion
    
$$ \hat p = \frac{\hat p_X N_X + \hat p_Y N_Y}{N_X + N_Y} = \frac{1_X + 1_Y}{N_X + N_Y} $$

$$s_p  = \sqrt{\hat p(1- \hat p)}$$ 

$$Z=\frac{\hat p_X-\hat p_Y}{\sqrt{\hat p(1-\hat p)(\frac{1}{N_X} + \frac{1}{N_Y})}} \sim N(0,1)$$
   - Option 2: Assume different proportion
$$Z=\dfrac{\hat p_X-\hat p_Y}{\sqrt{\dfrac{\hat p_X(1-\hat p_X)}{N_X}+\dfrac{\hat p_Y(1-\hat p_Y)}{N_Y}}}$$

**One-Tail vs. Two-Tail Test**
- Two Tail: Compare $|T|$ with $t_{m+n-2}
(\alpha/2) $ 
- One-Tail: Compare $T$ with $t_{m+n-2}(\alpha)$
<img src="https://saylordotorg.github.io/text_introductory-statistics/section_12/526c9e81a596b999ae191031b1b8bc47.jpg" width="400">

**Violation of assumptions**

- For $1^{st}$ assumption (Independence) : by experiment design

- For $2^{nd}$ assumption: Perform *Levene test *
    * Null hypothesis: samples have same variances
    * Reject null hypothesis when $p<\alpha=0.05$
    * When violated, the calculation of $df$ will change
    * Alternative: perform log transformation to stablize variation

- For 3rd assumption: Perform *Shapiro-Wilks test*
    - Reject null hypothesis when $p<\alpha=0.05$
    - When sample size is big, still valid (asymptotic normality)
    - Reason: Central Limit Theory for $\bar X$ and $\bar Y$

**Relationship with Likelihood-Ratio Test**
- Can be proved to be equivalent 

**Non-parametric methods**
- Mann-Whitney Test: https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test

- Wilcoxon Rank Sum Test

$$ {\displaystyle U_{1}=R_{1}-{n_{1}(n_{1}+1) \over 2}\,\!} $$

<img src="https://i2.wp.com/www.real-statistics.com/wp-content/uploads/2012/12/mann-whitney-test-excel.jpg" width="400">
- Advantages
    - Small sample size
    - Robust to outliers
    - No need for normal assumptions
- Disadvantages:
    - Higher Type II error
    - Lower power 
    - Not really needed for large sample

## Numerical Example for comparing proportions


In [61]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy
from scipy.stats import norm

Data source: https://github.com/nirupamaprv/Analyze-AB-test-Results

In [62]:
df = pd.read_csv('./data/ab_edited.csv')[:50]
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [63]:
old = df.query("group == 'control'")['converted']
new = df.query("group == 'treatment'")['converted']

### Implementation with `stats`

In [64]:
convert_old = sum(old)
convert_new = sum(new)
n_old = len(df.query("group == 'control'"))
n_new = len(df.query("group == 'treatment'"))

In [65]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], 
                                              [n_old, n_new], 
                                              prop_var = p_old)
z_score, p_value

(0.2556432050257977, 0.7982263538405426)

### Implementation with `numpy`

- Calculate $\bar{X}$ and $\bar{Y}$

In [66]:
mean_old = np.mean(old) 
mean_new = np.mean(new) 
mean_old, mean_new

(0.23809523809523808, 0.20689655172413793)

- Calculate $\hat \sigma_X$  and $\hat \sigma_Y$

In [67]:
s_old = s_new = np.sqrt(p_old * (1 - p_old))
s_old, s_new

(0.4259177099999599, 0.4259177099999599)

- Calculate $N_x$ and $N_y$

In [68]:
n_old = len(old) 
n_new = len(new) 
n_old, n_new

(21, 29)

- Calculate test statistics

In [69]:
Z = (mean_old - mean_new) / ( s_old * np.sqrt (1/n_old + 1/n_new) )
Z

0.2556432050257977

- Use normal distribution

In [70]:
from scipy.stats import norm
print(norm.cdf(Z))

0.6008868230797286


In [71]:
p = 1 - (norm.cdf(Z)-0.5) * 2
p

0.7982263538405427

# Numerical Example for comparing means

In [72]:
old=[79.98,80.04,80.02,80.04,80.03,80.03,80.04,79.97,80.05,80.03,80.02,80.00,80.02]
new=[80.02,79.94,79.98,79.97,79.97,80.03,79.95,79.97]

### Implementation with `stats`

In [90]:
t_score, p_value = scipy.stats.ttest_ind(old, new, equal_var = True)
t_score, p_value 

(3.472244847094969, 0.0025510042141041645)

### Implementation with `numpy`

- Perform *Shapiro-Wilks* Normality Test

In [74]:
from scipy.stats import shapiro
shapiro(old),shapiro(new)

((0.8718658089637756, 0.055402472615242004),
 (0.8912419080734253, 0.24032734334468842))

- **p-value**: larger than significance level: 0.05
- **Conclusion**: The normnal assumption is not violated 

- Perform *Levene* Test

In [75]:
from scipy.stats import levene
levene(old, new)

LeveneResult(statistic=0.215774685801598, pvalue=0.6475571540157947)

- **p-value**: larger than significance level: 0.05
- **Conclusion**: X and Y have same population variance

- Calculate $\bar{X}$ and $\bar{Y}$

In [91]:
mean_old = np.mean(old) 
mean_new = np.mean(new) 
mean_old, mean_new

(80.02076923076923, 79.97874999999999)

- Calculate $N_x$ and $N_y$

In [92]:
n_old = len(old) 
n_new = len(new) 
n_old, n_new

(13, 8)

- Calculate $S_x$ and $S_y$

In [93]:
s_old = np.std(old, ddof=1) 
s_new = np.std(new, ddof=1) 
s_old, s_new

(0.023965787580611863, 0.03136763573220897)

- Calculate DoF $df$

In [94]:
df1 = n_old - 1
df2 = n_new - 1
df = df1 + df2
df

19

- Calculate pooled variance

In [95]:
s_p = np.sqrt(( (n_old - 1) * s_old * s_old + (n_new - 1) * s_new * s_new) / (n_old + n_new - 2))
s_p

0.02693052239443681

- Calculate test statistics

In [96]:
T = (mean_old - mean_new) /(s_p * np.sqrt(1 / n_old + 1 / n_new))
T

3.472244847094969

- Use t distribution

In [97]:
from scipy.stats import t
t.cdf(T, df=df)

0.9987244978929479

In [98]:
p = 1 - (t.cdf(T, df=df)-0.5) * 2
p

0.0025510042141041467

### Calculate sample size / Power

**Power**: 
- $P$(reject $H_0$|$H_1$ is true)
- Commonly 80-95%
- Red shaded area

**What impacts power**
- Effect Size (+)
- Sample Size (+)
- Significant Level (e.g., 5%) (+)
- Population SD (-)
    - Conversion Rate vs. Actual number of visits
- ref: https://onlinecourses.science.psu.edu/stat414/node/304/

- Combined equation$$ES = (t_{\alpha/2} + t_{\beta})s_p\sqrt{\frac{1}{N_1}+\frac{1}{N_2}}\xrightarrow{\sigma_1 = \sigma_2, N_1 = N_2} = (t_{\alpha/2} + t_{\beta})\sigma\sqrt{\frac{2}{N}}$$



<img src="https://onlinecourses.science.psu.edu/stat414/sites/onlinecourses.science.psu.edu.stat414/files/lesson54/Lesson54_Drawing05/index.gif" width="300">


- Effect Size

<img src="https://onlinecourses.science.psu.edu/stat414/sites/onlinecourses.science.psu.edu.stat414/files/lesson54/Lesson54_Drawing08/index.gif" width="300">

<img src="https://onlinecourses.science.psu.edu/stat414/sites/onlinecourses.science.psu.edu.stat414/files/lesson54/Lesson54_Drawing09/index.gif" width="300">

- Significance Level

<img src="https://onlinecourses.science.psu.edu/stat414/sites/onlinecourses.science.psu.edu.stat414/files/lesson54/Lesson54_Drawing11/index.gif" width="300">

- Calculate sample size
<img src="https://onlinecourses.science.psu.edu/stat414/sites/onlinecourses.science.psu.edu.stat414/files/lesson54/Lesson54_Drawing16/index.gif" width="300">

In [101]:
p_baseline = 0.50 # under H_0
effect_size = 0.05 # Desired effect size
sig = 0.99
sample_size = 1001
#https://onlinecourses.science.psu.edu/stat414/node/306/

- Look up table: $Z(\alpha) = 2.326$


- Calculate power of test
    - Standardize user-provided $ES$ 
    $$ ES_{N(0,1)}  = \frac{ES}{ \sigma_{p = \bar X}} = \frac{ES}{\sigma_{X} / \sqrt{N}} = \frac{ES}{\sqrt{\frac{p(1-p)}{N}}} $$
$$$$
    - Calculate the arrow point on blue axis:
    $$ - ES_{N(0,1)} + Z_{critical}$$
    $$$$
    
    - Calculate the area of blue
    $$\phi_z(- ES_{N(0,1)} + Z_{critical})$$
    $$$$
    - Calculate the area of power
    $$1-\phi_z(- ES_{N(0,1)} + Z_{critical})$$



- How to calculate Sample Size: 
    - Formula for sample size estimation under $95\%$ significance and $80\%$ power.
$$N=\frac{16\sigma^2}{\Delta^2}$$
    

In [102]:
s_x = np.sqrt(p_baseline * (1 - p_baseline))
s_x

0.5

In [103]:
s_p =  s_x * np.sqrt( 1 / sample_size)
s_p

0.01580348853102535

In [104]:
effect_size_N_0_1 = effect_size / s_p
effect_size_N_0_1

3.163858403911275

In [105]:
phi_value = 2.326 - effect_size_N_0_1
phi_value

-0.8378584039112749

In [106]:
blue_shade = norm.cdf(phi_value)
blue_shade

0.2010551163605569

In [107]:
power = 1 - blue_shade
power

0.798944883639443

In [112]:
#Just use formula
N_size = 16 * p_baseline * (1 - p_baseline) / (effect_size * effect_size)
N_size

1599.9999999999998

# Online Experiment


## A/A Test
- Assign user to one of the two groups, but expose them to exactly same experience
- Calculate variability for power calculation
- Test the experimentation system (reject $H_0$ about 5% given significant level as 5%, with dummy treatments)
- Shut down treatment if significantly underperform

- Maybe something is wrong with how system assign treatment to users
<img src="https://cdn-images-1.medium.com/max/1600/0*wsfrJNYzgiPl41Tq.png" width="300"> 


## Type of variates
- Call to action
- Visual elements
- Buttons
- Text 
    - Ad: Promotion vs. Benefit vs. Information
    - Tweets: length/style/emoji/etc
- Image and Video
- Hashtags
- Backend (e.g., recommendation algorithm)

## Select evaluation metric
- Short-term vs. Long-term
    - adding more ads --> short-term revenue
    - loss of page views/clicks --> long-term revenue loss / user abandonment


- Consistent with long-term company goal, and sensitive enough to be detected
    - KPI: hard to detect change in a short time
    - Evaluation metric: strong correlation with KPI as a proxy


- **Example of metrics**:
    - Netflix Subscription: Viewing hours
    - Coursera Course certification: Test completion / Course Engagement


- ***By selecting better evaluation metric***

    - Search Engine: *Sessions per user* instead of *queries per user*
    - $\frac{Queries}{Month}=\frac{Queries}{Session}\times\frac{Session}{User}\times\frac{User}{Month}$


- ***By quantifying loss of traffic***:

    - Putting Ad on Homepage: *(decrease in click-through rate)* X *(visit frequency)* X *(cost of regenerating this traffic from other sources)*


## Limitations
- Analyze across key segments
    - Browser type
    - Device type
    - New and return users
    - Men and women
    - Geo
    - Age
    - Subscribers
    <img src="https://cdn-images-1.medium.com/max/1600/0*W-Qnxk2EQR3gzSeA.png" width="200">
    
    ***Alert***: Multiple comparison issue.
    
    
- Temporal factors (non-stationary time-series)
    - e.g, day of week effect
    - other periodical events
    
    
- Treatment ramp-up
    - Start from 0.1% treatment, gradually to 50%
    - A 50%/50% design is much faster than a 99%/1% design (25 times faster)


- Early stopping



- Preference to old or preference to newness
    - Novelty effect
    <img src="https://cdn-images-1.medium.com/max/1600/0*MlfgRSUftnrPE7wD.png" width="300">
    - Longer time
    - Only expose to new users
    
    
- Implementation cost
- Performance speed
    - Slow feature: bad experience

## AB test vs. Bandit

<img src="https://conductricsblog.files.wordpress.com/2012/08/abvbandit.jpg" width="500">
<img src="http://conductricsblog.files.wordpress.com/2012/08/aucbandit.jpg" width="500">

## Network effect

- **Sample consistency**: for example, GCP, two uses in one collaboration group faces two different features. Or adding a video chatting feature, which only works if both sides have access to it
- **Sample independency** (Spillover effect), for example, Facebook: many connected components, thus Group A and B are no longer independent.

- **Possible solution**: community (cluster) based AB test by partitioning nodes to groups, or for a dating app with no prior connections, maybe using demographic/geographical attributes
- Each ***cluster*** is assigned a treatment, thus unlikely for spillover from control to treatment
- Unit of analysis is reduced, **higher variance** as a result

Ref: http://web.media.mit.edu/~msaveski/projects/2016_network-ab-testing.html

<img src="http://web.media.mit.edu/~msaveski/assets/projects/2016_network-ab/main.png" width="500">


# Case Study

## Problem Statement
- Given a feature difference in facebook app, evaluate if the change will improve user activity.
- Given a UI component change (e.g., button color) in a pageview, evaluate if there are more users clicking.
- Given a pop-up message, whether users will continue enroll in program or not
- Given a new firewall feature in GCP

http://rajivgrover1984.blogspot.com/2015/11/ab-testing-overview.html
>*For example: An online education company tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course. If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that these courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.*

## Choose Subject (Unit of diversion)


Possible Choice: 
- User id
- Cookie
- Event

## Choose metric

Example of pop-up message and program enrollment

Guardrail Metrics that should **NOT** change:



- Number of cookies, unique # of cookies visiting the page
- Number of clicks on the button (since message shown after clicking)

Metrics that **MAY** change:


- User  Aquisition: $p = \frac{Number\ of\ users\ actually\ enrolled}{Number\ of\ users\ clicking\ button}$


- User Retention: $p = \frac{Number\ of\ users\ remain\ enrolled\ for\ 14\ days}{Number\ of\ users\ clicking\ button}$

# User Growth

<img src="http://zengzhangguan.com/wp-content/uploads/2018/08/aarrr-1-600x313-1-1.png" width="600">

<img src="https://static1.squarespace.com/static/55f6eebfe4b04a4eecaee07e/t/564391fde4b0ccf90c6f1852/1447268872573/" width="400">